In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import os
import cv2
from PIL import Image
import plotly.express as px
import plotly.graph_objects as go
from skimage.feature.texture import graycomatrix, graycoprops
from skimage.measure import label, regionprops
from skimage.measure import moments_hu
from tqdm import tqdm
from sklearn.cluster import DBSCAN
from scipy.stats import spearmanr
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from collections import defaultdict
import shutil
import hashlib

Définitions des fonctions statistiques et graphiques des caractéristiques

In [ ]:
df = pd.read_csv("/workspaces/datasciencetest_reco_plante/notebooks/Plant_V_Seg_all_features.csv")
df.head()

In [ ]:
# Analyse statistique - Violinplot interactif Plotly (distribution par classe)
def plot_violin_interactive(df, feature, classe='nom_plante'):
    fig = px.violin(df, y=feature, x=classe, box=True, points="all", hover_data=df.columns,
        title=f"Distribution de {feature} par {classe}")
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

In [ ]:
# Analyse statistique - Boxplot interactif Plotly
def plot_box_interactive(df, feature, classe='nom_plante'):
    fig = px.box(df, x=classe, y=feature, points="all", title=f"Boxplot de {feature} par {classe}",hover_data=df.columns)
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

In [ ]:
# Graphiques recommandés pour Objectif 2 : Déterminer si la plante est malade
# Comparer l’état (Est_Saine) pour les features de couleur/texture

plot_box_interactive(df, feature='contrast', classe='Est_Saine')
plot_violin_interactive(df, feature='mean_R', classe='Est_Saine')
plot_box_interactive(df, feature='energy', classe='Est_Saine')